# Testing on Driving Data

In [1]:
from dataset import PresampledDataset
from model import *

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
trajs = pd.read_pickle("data/trajectories_slim.pkl")

In [3]:
# build traj dataset 
traj_mat = np.zeros((1106,33,8))
traj_mat_p = np.zeros((1106,33,8))
for i,label in enumerate(['x1', 'y1', 'xd1', 'yd1', 'x2', 'y2','xd2', 'yd2']):
    itr = -1
    for k,v in trajs[label].items():
        if k[1] == 0:
            itr += 1
        if k[1] < 33:
            traj_mat[itr,k[1],i] = v
        if (not (k[1] == 0)) and k[1]<34:
            traj_mat_p[itr,k[1]-1,i] = v

#shuffle this data
inds = np.random.choice(traj_mat.shape[0],traj_mat.shape[0])
traj_mat = traj_mat[inds,:,:]
traj_mat_p = traj_mat_p[inds,:,:]
            
N_train = 1000
N_test = 100
X = traj_mat[:1000,:,:]
Y = traj_mat_p[:1000,:,:] #- X

X_test = traj_mat[1000:1100,:,:]
Y_test = traj_mat_p[1000:1100,:,:] #- X_test

In [4]:
dataset = PresampledDataset(X,Y)

## Create conditional model and train it on the data

In [49]:
config = {'num_bijectors': 4,
          'bijector': 'RealNVP',
          'network_size': 64,
          'use_batchnorm': True,
          'y_dim': 8,
          'x_dim': 8,
          'nn_layers': [64,64,32],
          'activation': 'tanh',
          'sigma_eps': 1.0,
          'batch_size': 50,
          'data_horizon': 0,
          'test_horizon': 10,
          'learning_rate': 1e-3}

In [50]:
g1 = tf.Graph()
sess1 = tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=g1)

In [51]:
model = CondInvicuna(sess1, config)
model.construct_model()

In [54]:
model.train(dataset, 5000)

itr: 0 ; loss: 65.47651
itr: 20 ; loss: 19.13189
itr: 40 ; loss: 15.537289
itr: 60 ; loss: 12.78259
itr: 80 ; loss: 12.883328
itr: 100 ; loss: 11.524578
itr: 120 ; loss: 12.472013
itr: 140 ; loss: 12.037472
itr: 160 ; loss: 11.298044
itr: 180 ; loss: 10.817397
itr: 200 ; loss: 11.758363
itr: 220 ; loss: 11.222841
itr: 240 ; loss: 10.3872595
itr: 260 ; loss: 9.958813
itr: 280 ; loss: 9.884608
itr: 300 ; loss: 10.183756
itr: 320 ; loss: 10.767787
itr: 340 ; loss: 10.371936
itr: 360 ; loss: 10.0637245
itr: 380 ; loss: 10.97841
itr: 400 ; loss: 8.736973
itr: 420 ; loss: 8.3255825
itr: 440 ; loss: 9.322081
itr: 460 ; loss: 8.5352955
itr: 480 ; loss: 7.855633
itr: 500 ; loss: 8.23634
itr: 520 ; loss: 7.2971087
itr: 540 ; loss: 7.9938693
itr: 560 ; loss: 8.322334
itr: 580 ; loss: 7.587249
itr: 600 ; loss: 7.7521315
itr: 620 ; loss: 7.843273
itr: 640 ; loss: 7.767425
itr: 660 ; loss: 7.9485383
itr: 680 ; loss: 7.898918
itr: 700 ; loss: 6.9464436
itr: 720 ; loss: 7.185412
itr: 740 ; loss: 7.086

KeyboardInterrupt: 

Evaluate test likelihood of prior

In [60]:
np.mean(np.log(model.get_likelihood(X_test[:50,0:0,:],Y_test[:50,0:0,:],X_test[:50,:,:],Y_test[:50,:,:])))

-6.125162